In [ ]:
wd<-"/result/RCTD/ref_10x_astr/L3_taxon/Astrocyte_integrate_split2A/"
#dir.create(wd)
setwd(wd)
getwd()

In [ ]:
library(Seurat)
library(ggplot2)
library(patchwork)
library(dplyr)
#library(hdf5r)
library(png)
library(ggalluvial)
#library(tidyverse)
#library(clustree)
library(RCTD)
library(Matrix)
#library(SPOTlight)
library(future)
library(grid)
library(RColorBrewer)
#library(pheatmap)
library(circlize)
library(igraph)
#library(ComplexHeatmap)
library(stringr)
#library(ComplexHeatmap)
library(cowplot)
library(clusterProfiler)
options(connectionObserver = NULL)
library(biomaRt)
#library(scClustViz)

In [ ]:
### 10X reference
#da<-readRDS("/home/jovyan/SCI_visium/ref_deconvolution/GSE162610/SCI.integrated.20210830.rds")
#da
me<-read.csv("/result/RCTD/10Xreference_myeloid_recluster/obs_sci.csv")
da<-readRDS("/data/deconvolution/sci.rds")
da

In [ ]:
DefaultAssay(da)<-"RNA"
da

In [ ]:
cell<-intersect(me$X,colnames(da))
length(cell)

In [ ]:
rownames(me)<-me$X
me<-me[,-1]
me<-me[colnames(da),]
table(colnames(da)==rownames(me))

In [ ]:
neuron<-rownames(da@meta.data[da@meta.data$celltype=="Neuron",])
length(neuron)

In [ ]:
me[neuron,"celltype"]<-"Neuron"
table(me$celltype)
me[neuron,"L1_taxon"]<-"Neuron"
table(me$L1_taxon)
me[neuron,"L2_taxon"]<-"Neuron"
table(me$L2_taxon)
me[neuron,"L3_taxon"]<-"Neuron"
table(me$L3_taxon)

In [ ]:
na_cell<-rownames(me[is.na(me$L3_taxon),])
length(na_cell)

In [ ]:
me_rm<-me[!rownames(me)%in%na_cell,]
dim(me_rm)

In [ ]:
da_rm<-da[rownames(da),rownames(me_rm)]
da_rm

In [32]:
da_rm@meta.data<-me_rm

In [33]:
da_ad<-da_rm

In [34]:
saveRDS(da_rm,"10X.L3_taxon.na.processed.rds")

In [7]:
da_rm<-readRDS("10X.L3_taxon.na.processed.rds")

In [36]:
DefaultAssay(da_rm)<-"RNA"

In [37]:
da_rm<-CreateSeuratObject(counts = da_rm@assays$RNA@counts,meta.data = da_rm@meta.data)

In [ ]:
da1_astr<-da_rm[rownames(da_rm),rownames(da_rm@meta.data[da_rm@meta.data$L3_taxon=="Astrocyte",])]
da1_astr
table(da1_astr$L3_taxon)

In [40]:
saveRDS(da1_astr,"10X.L3.Astrcoyte.RNA.rds")

In [1]:
### split-pool reference
da2<-readRDS("/result/RCTD/split_ref/sc.reference.ALS.rmUnkonwn.rds")


In [ ]:
meta2<-data.frame(da2@cell_types,row.names = colnames(da2@counts))
dim(meta2)
colnames(meta2)[1]<-"cell_types"
table(meta2$cell_types)

In [ ]:
da_s<-CreateSeuratObject(counts = da2@counts,meta.data = meta2
                        )
da_s

In [7]:
da_s<-NormalizeData(da_s)

In [8]:
Idents(da_s)<-da_s$cell_types

In [ ]:
pdf("Rosenberg.allcelltypes.Pla2g7.Vlnplot_2.pdf",width = 10,height = 4)
options(repr.plot.width=10,repr.plot.height=4)
VlnPlot(da_s,features = c("Pla2g7"),#ncol = 4,
        #cols = c('#911eb4',"#ff7b25","#e6beff"),
        #slot = "scale.data",
        pt.size = 0)
dev.off()

In [ ]:
pdf("Rosenberg.allcelltypes.Igfbp2.Vlnplot_1.pdf",width = 10,height = 4)
options(repr.plot.width=10,repr.plot.height=4)
VlnPlot(da_s,features = c("Igfbp2"),#ncol = 4,
        #cols = c('#911eb4',"#ff7b25","#e6beff"),
        #slot = "scale.data",
        pt.size = 0.5)
dev.off()

In [9]:
# Basic function to convert human to mouse gene names
convertHumanGeneList <- function(x){

require("biomaRt")
human = useMart("ensembl", dataset = "hsapiens_gene_ensembl")
mouse = useMart("ensembl", dataset = "mmusculus_gene_ensembl")

genesV2 = getLDS(attributes = c("hgnc_symbol"), filters = "hgnc_symbol", values = x , mart = human, attributesL = c("mgi_symbol"), martL = mouse, uniqueRows=T)

humanx <- unique(genesV2[, 2])

# Print the first 6 genes found to the screen
print(head(humanx))
return(humanx)
}

In [ ]:
### signatures of cell types
cell_marker<-FindAllMarkers(da_s,logfc.threshold = 1,only.pos = TRUE)

In [16]:
write.csv(cell_marker,"Rosenberg.celltype_marker.fc1.pos.csv")

In [ ]:
da2_astr<-da_s[rownames(da_s),rownames(da_s@meta.data[da_s@meta.data$cell_types %in% c('Astrocyte_Gfap','Astrocyte_Slc7a10',"Astro_Svep1"),])]
da2_astr
table(da2_astr$cell_types)

In [64]:
saveRDS(da2_astr,"split-pool.astrocyte.rds")

### label transfer split-pool astrocyte

In [67]:
query<-da1_astr
query<-NormalizeData(query,verbose = F)
query<-FindVariableFeatures(query,selection.method = "vst")
query<-ScaleData(query,verbose=F)

In [71]:
ref<-da2_astr
ref<-NormalizeData(ref,verbose = F)
ref<-FindVariableFeatures(ref,selection.method = "vst")
ref<-ScaleData(ref,verbose = F)

In [ ]:
ref<-RunPCA(ref,npcs = 30,verbose = F)
ref<-RunUMAP(ref,reduction = "pca",dims = 1:30)
#DimPlot(ref,reduction = "umap",group.by = "cell_types")

In [ ]:
anchors<-FindTransferAnchors(reference = ref,query = query,dims = 1:30)
predictions<-TransferData(anchorset=anchors,refdata=ref$cell_types,dims = 1:30)
query<-AddMetaData(query,metadata = predictions)

In [ ]:
options(repr.plot.width=10,repr.plot.height=6)
VlnPlot(query,c("Gfap","Slc7a10","Svep1"),group.by = "predicted.id",pt.size = 0)
ggsave("splitpool_3astr.labeltransfer.10x_L3.astr.markers.vlnplot_220509.png",width = 10,height = 6)

In [80]:
cell1<-rownames(query@meta.data[query@meta.data$predicted.id=="Astro_Svep1",])
cell2<-rownames(query@meta.data[query@meta.data$predicted.id=="Astrocyte_Gfap",])
cell3<-rownames(query@meta.data[query@meta.data$predicted.id=="Astrocyte_Slc7a10",])

In [87]:
da3<-da2[rownames(da2),#rownames(da2@meta.data[!da2@meta.data$time=="7dpi",])
        ]
da3$labeltranfer<-da3$L3_taxon
da3@meta.data[cell1,"labeltranfer"]<-"Astro_Svep1"
da3@meta.data[cell2,"labeltranfer"]<-"Astrocyte_Gfap"
da3@meta.data[cell3,"labeltranfer"]<-"Astrocyte_Slc7a10"

In [ ]:
rmcell<-names(da3$labeltranfer[!is.na(da3$labeltranfer)])
length(rmcell)
head(rmcell)
da3_filt<-da3[rownames(da3),rmcell]
unique(da3_filt$labeltranfer)

In [ ]:
table(da3_filt$labeltranfer)
Idents(da3_filt)<-da3_filt$labeltranfer
levels(Idents(da3_filt))

In [ ]:
options(repr.plot.width=15,repr.plot.height=15)
VlnPlot(da3_filt,c("Gfap","Slc7a10"),group.by = "labeltranfer",pt.size = 0,ncol = 1)
ggsave("splitpool_3astr.labeltransfer.10x_L3.astrocyte.marker.across.allcelltypes.vlnplot_220509.png",width=15,height=15)


In [94]:
saveRDS(da3_filt,"splitpool_3astr.labeltransfer.10x_L3.astrocyte.with7dpi.rds")

In [95]:
## subset astrocyte object
da3_astr<-da3_filt[rownames(da3_filt),rownames(da3_filt@meta.data[da3_filt@meta.data$labeltranfer %in% c('Astrocyte_Slc7a10','Astrocyte_Gfap','Astro_Svep1'),])]
saveRDS(da3_astr,"splitpool_3astr.labeltransfer.10x_L3.astrocyte.labelAstrcoyte.object.with7dpi.rds")

In [ ]:
set.seed(1234)
da3_filt_down<-subset(da3_filt,downsample=1000)
table(Idents(da3_filt_down))

In [ ]:
options(repr.plot.width=15,repr.plot.height=15)
VlnPlot(da3_filt_down,c("Gfap","Slc7a10"),group.by = "labeltranfer",pt.size = 0,ncol = 1)
ggsave("2astr.labeltransfer.L3.astrocyte.marker.across.allcelltypes.without7dpi.downsample1000.vlnplot.png",width=15,height=15)

In [ ]:
## create RCTD reference
table(Idents(da3_filt_down))
cell<-da3_filt_down$labeltranfer
cell<-as.factor(cell)
head(cell)
reference<-Reference(da3_filt_down@assays$RNA@counts,cell)
saveRDS(reference,"2reference.2astr.labeltransfer.L3.astrocyte_RCTD.reference.seed1234.down1000.without7dpi.rds")